# Forecast Performance Visualization

This notebook analyzes the performance of the AI stock analysis agent by comparing its recommendations against actual market performance.

In [ ]:
import sqlite3
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import sys
import contextlib
import io

# --- Path Setup ---
current_dir = os.getcwd()
project_root = None
for _ in range(3):
    if os.path.exists(os.path.join(current_dir, 'app')):
        project_root = current_dir
        break
    current_dir = os.path.dirname(current_dir)

if project_root:
    sys.path.insert(0, project_root)
    print(f"Project Root: {project_root}")
    DB_PATH = os.path.join(project_root, "subscribers.db")
else:
    DB_PATH = "../subscribers.db" # Fallback
    print("Project Root not found, using fallback DB path")

# Setup Plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
def fetch_decisions():
    conn = sqlite3.connect(DB_PATH)
    query = """
    SELECT 
        id, timestamp, symbol, region, ai_score, recommendation, 
        price_at_decision, deep_research_verdict
    FROM decision_points
    WHERE timestamp >= date('now', '-90 days')
    ORDER BY timestamp DESC
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

df = fetch_decisions()
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date_str'] = df['timestamp'].dt.strftime('%Y-%m-%d')

# Deduplication: Keep latest decision per symbol per day
df = df.sort_values(['symbol', 'timestamp'], ascending=[True, False])
df = df.drop_duplicates(subset=['symbol', 'date_str'], keep='first')

print(f"Loaded {len(df)} decisions.")
df.head()

In [ ]:
# Batch Fetch Price Data

def get_current_prices(symbols):
    print(f"Fetching data for {len(symbols)} symbols...")
    # Note: Using simple yfinance download. For complex tickers, resolver logic from app might be needed.
    # To keep notebook self-contained, we assume standard tickers or simple mappings.
    
    # Chunking to prevent URL length issues
    chunk_size = 100
    all_data = pd.DataFrame()
    
    for i in range(0, len(symbols), chunk_size):
        chunk = symbols[i:i+chunk_size]
        try:
            data = yf.download(chunk, period="3mo", progress=False)['Close']
            if isinstance(data, pd.Series): # Handle single ticker case
                 data = data.to_frame()
            
            if all_data.empty:
                all_data = data
            else:
                all_data = pd.concat([all_data, data], axis=1)
        except Exception as e:
            print(f"Error fetching chunk {i}: {e}")
            
    return all_data

symbols = df['symbol'].unique().tolist()
price_data = get_current_prices(symbols)
print("Price data fetched.")

In [ ]:
# Calculate Performance

results = []

for idx, row in df.iterrows():
    symbol = row['symbol']
    start_price = row['price_at_decision']
    date_str = row['date_str']
    decision_date = row['timestamp']
    
    # Target: +1 Week
    target_date = decision_date + timedelta(days=7)
    
    current_price = None
    outcome_price = None
    status = "Pending"
    
    if symbol in price_data.columns:
        hist = price_data[symbol].dropna()
        
        # Get Latest Price
        if not hist.empty:
            current_price = hist.iloc[-1]
            
        # Get Approx 1 Week Price
        # Find first index >= target_date
        future_mask = hist.index >= target_date.tz_localize(hist.index.tz)
        future_hist = hist[future_mask]
        
        if not future_hist.empty:
            outcome_price = future_hist.iloc[0]
            status = "Completed"
        elif datetime.now() > target_date:
             # Date passed but no data (delisted? or gap)
             # Use current if available
             if current_price:
                 outcome_price = current_price
                 status = "Completed (Fallback)"

    # Perf Calc
    roi = 0.0
    used_price = outcome_price if outcome_price else current_price
    
    if start_price and used_price:
        roi = ((used_price - start_price) / start_price) * 100
        
    results.append({
        'id': row['id'],
        'symbol': symbol,
        'date': date_str,
        'rec': row['recommendation'],
        'score': row['ai_score'],
        'roi': roi,
        'status': status
    })

res_df = pd.DataFrame(results)
res_df.head()

In [ ]:
# Visualizations

# 1. Performance Distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=res_df, x='roi', hue='rec', kde=True, bins=30)
plt.title('Return on Investment Distribution by Recommendation')
plt.xlabel('ROI (%)')
plt.xlim(-50, 50) # Limit outliers for view
plt.show()

In [ ]:
# 2. ROI by Recommendation Type (Boxplot)
plt.figure(figsize=(10, 6))
sns.boxplot(data=res_df, x='rec', y='roi')
plt.title('ROI Performance by Recommendation')
plt.ylim(-50, 50)
plt.show()

In [ ]:
# 3. Average ROI by Recommendation
avg_roi = res_df.groupby('rec')['roi'].mean().sort_values()
print("Average ROI by Recommendation:")
print(avg_roi)

avg_roi.plot(kind='bar', color='skyblue')
plt.title('Average ROI by Rec')
plt.ylabel('Avg ROI (%)')
plt.show()